Importation des bibli 

In [7]:
import pandas as pd
import re
import nltk
from tqdm import tqdm
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

nltk.download('stopwords')
nltk.download('punkt')  # Nécessaire pour tokenizer les mots si besoin

stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ali_nhs/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/ali_nhs/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Lancement de la db avec que les colonnes importants

In [3]:
# Charger les données à partir du fichier CSV
dataBrevet = pd.read_csv('brevet.csv')

# Supprimer les colonnes inutiles pour la catégorisation
columns_to_keep = ['claim', 'description', 'CPC']
dataBrevet = dataBrevet[columns_to_keep]


# Fusionner les colonnes 'claim' et 'description'
dataBrevet['FusionClaimDescription'] = dataBrevet['claim'] + ' ' + dataBrevet['description']

# Supprimer les colonnes 'claim' et 'description'
dataBrevet.drop(['claim', 'description'], axis=1, inplace=True)

# Afficher les premières lignes pour vérifier la structure des données échantillonnées
print(dataBrevet.head())

KeyboardInterrupt: 

Définir les Fonctions de Nettoyage
Définir les Fonctions de Nettoyage
Définir les Fonctions de Nettoyage

In [35]:
# Fonction pour nettoyer les balises XML
def clean_text(text):
    if pd.isna(text):
        return ""
    soup = BeautifulSoup(text, 'lxml')
    return soup.get_text()

# Fonction pour nettoyer la ponctuation, convertir en minuscules, etc.
def basic_cleaning(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return text

# Fonction pour nettoyer les stopwords
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)
    
# Fonction pour appliquer le stemming
def apply_stemming(text):
    words = text.split()
    stemmed_words = [stemmer.stem(word) for word in words]
    return ' '.join(stemmed_words)



In [40]:

tqdm.pandas()

# Nettoyer la colonne fusionnée
dataBrevet['FusionClaimDescription'] = (
    dataBrevet['FusionClaimDescription']
    .progress_apply(apply_stemming)
    .progress_apply(clean_text)
    .progress_apply(basic_cleaning)
    .progress_apply(remove_stopwords)
    
)


# Afficher les données nettoyées
print(dataBrevet[['FusionClaimDescription', 'CPC']].head())

100%|███████████████████████████████████| 50000/50000 [00:47<00:00, 1043.71it/s]


                              FusionClaimDescription  \
0  method manufactur cover window display panel c...   
1  compress screw appli compress bone joint compr...   
2  cloth dryer 100 comprisinga case 4a dri drum 1...   
3  diplex comprisinga filter pair constitut first...   
4  method display electron program guide epg user...   

                                                 CPC  
0  ['B23K26-361', 'H04M1-0266', 'B23K26-402', 'G0...  
1  ['A61B17-68', 'A61B17-888', 'A61B17-8685', 'A6...  
2                        ['D06F58-206', 'D06F58-20']  
3  ['H01P1-2138', 'H01P5-107', 'H01P1-2088', 'H01...  
4      ['H04N21-4345', 'H04N21-4821', 'H04N21-4316']  


In [5]:
# Sauvegarder les données nettoyées dans un fichier CSV
dataBrevet.to_csv('brevet_cleaned.csv', index=False)

NameError: name 'dataBrevet' is not defined

In [11]:
# Charger les données nettoyées
dataBrevet_cleaned = pd.read_csv('brevet_cleaned.csv')
dataBrevet_cleaned.head()

,CPC,FusionClaimDescription
0,"['B23K26-361', 'H04M1-0266', 'B23K26-402', 'G0...",method manufactur cover window display panel c...
1,"['A61B17-68', 'A61B17-888', 'A61B17-8685', 'A6...",compress screw appli compress bone joint compr...
2,"['D06F58-206', 'D06F58-20']",cloth dryer 100 comprisinga case 4a dri drum 1...
3,"['H01P1-2138', 'H01P5-107', 'H01P1-2088', 'H01...",diplex comprisinga filter pair constitut first...
4,"['H04N21-4345', 'H04N21-4821', 'H04N21-4316']",method display electron program guide epg user...


In [19]:
dataBrevet_cleaned.head(6)

,CPC,FusionClaimDescription
0,"['B23K26-361', 'H04M1-0266', 'B23K26-402', 'G0...",method manufactur cover window display panel c...
1,"['A61B17-68', 'A61B17-888', 'A61B17-8685', 'A6...",compress screw appli compress bone joint compr...
2,"['D06F58-206', 'D06F58-20']",cloth dryer 100 comprisinga case 4a dri drum 1...
3,"['H01P1-2138', 'H01P5-107', 'H01P1-2088', 'H01...",diplex comprisinga filter pair constitut first...
4,"['H04N21-4345', 'H04N21-4821', 'H04N21-4316']",method display electron program guide epg user...
5,"['F16D41-22', 'F16D43-22', 'F16D43-06', 'F16D1...",clutch assembl comprisingan output compon 12 i...


In [13]:
!pip install fasttext

In [15]:
# Extraire toutes les premières lettres de chaque catégorie CPC
def extract_first_letters(cpc_list):
    return list(set([code[0] for code in eval(cpc_list)]))

dataBrevet_cleaned['first_letters'] = dataBrevet_cleaned['CPC'].apply(extract_first_letters)
dataBrevet_cleaned['first_letters']


0        [G, B, H]
1              [A]
2              [D]
3              [H]
4              [H]
           ...    
49995       [C, A]
49996          [B]
49997       [C, A]
49998       [G, H]
49999    [Y, B, F]
Name: first_letters, Length: 50000, dtype: object

In [21]:
from sklearn.model_selection import train_test_split


# Diviser les données en ensembles d'entraînement et de test
train_data, test_data = train_test_split(dataBrevet_cleaned, test_size=0.3, random_state=1)

# Fonction pour écrire les données dans un fichier au format FastText
def write_fasttext_file(filename, data):
    with open(filename, 'w') as f:
        for _, row in data.iterrows():
            labels = " ".join([f"__label__{label}" for label in row['first_letters']])
            f.write(f"{labels} {row['FusionClaimDescription']}\n")

# Préparer les fichiers pour FastText
write_fasttext_file('train_first_letter.txt', train_data)
write_fasttext_file('test_first_letter.txt', test_data)

Lecture et Prétraitement des Données : Chargement et tokenisation des données d'entraînement.
Construction du Vocabulaire et des N-grammes : Création du vocabulaire et génération des n-grammes.
Initialisation des Poids et des Embeddings : Initialisation des vecteurs de mots et des poids du modèle.

In [25]:
import fasttext

# Entraîner le modèle FastText pour la première lettre des catégories CPC
model_first_letter = fasttext.train_supervised(
    input='train_first_letter.txt',
    epoch=100, 
    lr=0.5, 
    wordNgrams=5,  
    bucket=200000,
    dim=100,
    loss='softmax'
)
# Sauvegarder le modèle
model_first_letter.save_model("model_first_letter.bin")

Read 219M words
Number of words:  2525217
Number of labels: 9
Progress: 100.0% words/sec/thread: 1433877 lr:  0.000000 avg.loss:  0.450028 ETA:   0h 0m 0s  0h35m41s ETA:   0h35m 3s 0.478638 avg.loss:  1.201339 ETA:   0h35m 2s lr:  0.477814 avg.loss:  1.188484 ETA:   0h34m55s  0h33m37s 15.8% words/sec/thread: 1484031 lr:  0.420841 avg.loss:  0.786143 ETA:   0h29m35s 20.6% words/sec/thread: 1467179 lr:  0.396980 avg.loss:  0.718417 ETA:   0h28m14ss 37.8% words/sec/thread: 1490344 lr:  0.310851 avg.loss:  0.592389 ETA:   0h21m45s39s 38.7% words/sec/thread: 1483876 lr:  0.306330 avg.loss:  0.588434 ETA:   0h21m32s  0h21m23s 39.6% words/sec/thread: 1477856 lr:  0.302007 avg.loss:  0.584547 ETA:   0h21m19s1473123 lr:  0.288600 avg.loss:  0.572880 ETA:   0h20m26s1472740 lr:  0.281996 avg.loss:  0.567893 ETA:   0h19m58s% words/sec/thread: 1473154 lr:  0.174432 avg.loss:  0.506604 ETA:   0h12m21s% words/sec/thread: 1472926 lr:  0.173240 avg.loss:  0.505995 ETA:   0h12m16s 68.4% words/sec/thread

Précision : Si votre modèle prédit 100 brevets comme étant de catégorie "A", mais que seulement 80 d'entre eux sont effectivement de catégorie "A", alors la précision est de 80%.
Rappel : Si en réalité, il y a 100 brevets de catégorie "A", mais que votre modèle n'en a identifié que 80, alors le rappel est de 80%.
Score F1 : Le score F1 pour ces valeurs de précision et de rappel serait de 80%.


In [46]:
# Charger le modèle
model_first_letter = fasttext.load_model("model_first_letter.bin")

# Évaluer le modèle sur l'ensemble de test
results = model_first_letter.test('test_first_letter.txt')

# Extraire les métriques de performance
num_examples, precision, recall = results
print(f"Nombre d'exemples : {num_examples}")
print(f"Précision : {precision:.4f}")
print(f"Rappel : {recall:.4f}")

# Calculer le F1-score
f1_score = 2 * (precision * recall) / (precision + recall)
print(f"F1-score : {f1_score:.4f}")


Nombre d'exemples : 15000
Précision : 0.8664
Rappel : 0.5875
F1-score : 0.7002


	•	Précision (Precision) : 0.8631
	•	La précision mesure la proportion de vraies prédictions positives parmi toutes les prédictions positives. Une précision de 0.8631 signifie que 86.31% des étiquettes prédites sont correctes.
	•	Rappel (Recall) : 0.5853
	•	Le rappel mesure la proportion de vraies prédictions positives parmi toutes les instances positives réelles. Un rappel de 0.5853 signifie que le modèle récupère 58.53% des étiquettes positives réelles.
	•	F1-score : 0.7002
	•	Le F1-score est la moyenne harmonique de la précision et du rappel. Il est particulièrement utile lorsque vous avez besoin d’un équilibre entre précision et rappel.

	•	Précision élevée, mais rappel relativement faible :
	•	Cela signifie que, bien que le modèle fasse peu d’erreurs lorsqu’il fait une prédiction positive (haute précision), il manque beaucoup d’instances positives réelles (faible rappel).

In [31]:
# Exemple de texte pour la prédiction après le nettoyage et la fusion des colonnes
sample_text = dataBrevet_cleaned.iloc[20000]['FusionClaimDescription'][:1000]
cpc_sample = dataBrevet_cleaned.iloc[20000]['CPC']
print("FusionClaimDescription:", sample_text)
print("CPC:", cpc_sample)


FusionClaimDescription: tinplat copper termin materi comprising substrat made copper copper alloy intermedi zinc layer form substrate made zinc alloy ha thick less 010 500 tin layer made tin tin alloy form intermedi zinc layer proport length occupi lowangl grain boundari less 2 30 respect total length crystal grain boundariesth tinplat copper termin materi accord claim 1 wherein corros potenti respect silversilv chlorid electrod 500 mv less 900 mvth tinplat copper termin materi accord claim 1 wherein intermedi zinc layer compris one among nickel iron manganese molybdenum cobalt cadmium lead addit element content percentag zinc intermedi zinc layer less 65 mass 95 massth tinplat copper termin materi accord claim 1 wherein averag crystal grain size tin layer less 05 80 mth tinplat copper termin materi accord claim 1 wherein surfac metalliczinc layer provid tin layerth tinplat copper termin materi accord claim 5 wherein surface metalliczinc layer ha zinc concentr less 5 40 thick less 10 n

In [37]:
# Exemple donnée dans le notion
cleaned_text = clean_text(sample_text)
cleaned_text = basic_cleaning(cleaned_text)
cleaned_text = remove_stopwords(cleaned_text)

# Prédire les premières lettres avec les 1 labels les plus probables
predictions = model_first_letter.predict(cleaned_text, k=2)

# Extraire les labels prédits et les nettoyer
predicted_first_letters = [label.replace("__label__", "") for label in predictions[0]]
predicted_probabilities = predictions[1]
print(f"Predicted first letters: {', '.join(predicted_first_letters)}")
print(f"Predicted probabilities: {', '.join(map(str, predicted_probabilities))}")

Predicted first letters: H, C
Predicted probabilities: 0.5075941681861877, 0.44304996728897095


In [55]:

# Fonction pour expliquer la prédiction
def explain_prediction(model, text, top_k=2, threshold=0.01, top_n=10):
    # Prédiction initiale
    initial_predictions = model.predict(text, k=top_k)
    initial_probs = initial_predictions[1]

    words = text.split()
    word_importances = {label: [] for label in initial_predictions[0]}
    
    for i, word in enumerate(words):
        perturbed_text = " ".join(words[:i] + words[i+1:])
        perturbed_predictions = model.predict(perturbed_text, k=top_k)
        perturbed_probs = perturbed_predictions[1]

        for j in range(top_k):
            label = initial_predictions[0][j]
            importance = initial_probs[j] - perturbed_probs[j]
            if importance >= threshold:
                word_importances[label].append((word, importance))

    # Trier les mots par importance pour chaque label et garder les top_n mots
    for label in word_importances:
        word_importances[label] = sorted(word_importances[label], key=lambda x: x[1], reverse=True)[:top_n]

    return word_importances


# Prédire les premières lettres avec les labels les plus probables
predictions = model_first_letter.predict(cleaned_text, k=2)

# Extraire les labels prédits et les nettoyer
predicted_first_letters = [label.replace("__label__", "") for label in predictions[0]]
predicted_probabilities = predictions[1]
print(f"Predicted first letters: {', '.join(predicted_first_letters)}")
print(f"Predicted probabilities: {', '.join(map(str, predicted_probabilities))}")

# Expliquer la prédiction
word_importances = explain_prediction(model_first_letter, cleaned_text, top_k=2, threshold=0.01, top_n=10)

# Afficher les mots importants pour chaque lettre prédite
print("\nWords that influenced the prediction:")
for label in word_importances:
    print(f"\nLabel: {label}")
    for word, importance in word_importances[label]:
        print(f"  Word: {word}, Impact: {importance:.4f}")


Predicted first letters: H, C
Predicted probabilities: 0.5075941681861877, 0.44304996728897095

Words that influenced the prediction:

Label: __label__H
  Word: accord, Impact: 0.0458
  Word: sio2th, Impact: 0.0398
  Word: accord, Impact: 0.0360
  Word: one, Impact: 0.0343
  Word: tin, Impact: 0.0340
  Word: 5, Impact: 0.0331
  Word: 1, Impact: 0.0326
  Word: iron, Impact: 0.0305
  Word: less, Impact: 0.0297
  Word: accord, Impact: 0.0281

Label: __label__C
  Word: 500, Impact: 0.1797
  Word: addit, Impact: 0.1787
  Word: percentag, Impact: 0.1676
  Word: element, Impact: 0.1513
  Word: compris, Impact: 0.1291
  Word: materi, Impact: 0.1288
  Word: materi, Impact: 0.1269
  Word: zinc, Impact: 0.1194
  Word: materi, Impact: 0.1187
  Word: tin, Impact: 0.1168


In [148]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# Fonction pour obtenir les étiquettes prédictes pour chaque ligne du fichier de test
def get_predicted_labels(model, test_file):
    true_labels = []
    predicted_labels = []
    with open(test_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            true_label = parts[0].replace("__label__", "")
            text = " ".join(parts[1:])
            prediction = model.predict(text, k=1)
            predicted_label = prediction[0][0].replace("__label__", "")
            true_labels.append(true_label)
            predicted_labels.append(predicted_label)
    return true_labels, predicted_labels

# Obtenir les étiquettes véritables et prédictes
true_labels, predicted_labels = get_predicted_labels(model_first_letter, 'test_first_letter.txt')

# Convertir les étiquettes en numpy arrays
true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

# Générer la matrice de confusion
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Générer le rapport de classification
class_report = classification_report(true_labels, predicted_labels)

# Afficher la matrice de confusion et le rapport de classification
print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)

Confusion Matrix:
[[1145   98   41    8    7   43  101   33    3]
 [ 109 1351   34    8   19  185   87  111   16]
 [ 234  221 1338   16    3   42   75  138   64]
 [  19   37   14  101    3   11    6    1    2]
 [  28   84    1    0  146   53   16    8    0]
 [  36   60    3    1   11  644   12   57   19]
 [ 174  285   23    4   12   69 2939  664   52]
 [  18   38    9    0    4   20  177 2192   24]
 [  41  243   28    5    9  287   31  471  278]]

Classification Report:
              precision    recall  f1-score   support

           A       0.63      0.77      0.70      1479
           B       0.56      0.70      0.62      1920
           C       0.90      0.63      0.74      2131
           D       0.71      0.52      0.60       194
           E       0.68      0.43      0.53       336
           F       0.48      0.76      0.59       843
           G       0.85      0.70      0.77      4222
           H       0.60      0.88      0.71      2482
           Y       0.61      0.20     

En vrai c'est pas mal, 0,7 de f1-score pas mal pour la première lettre 

In [ ]:
MAINTENANT ON VA PASSER AU DEUX DIGIT APRÈS LA LETTRE 

In [39]:
import os

# Fonction pour extraire les deux chiffres après la première lettre de chaque catégorie CPC
def extract_first_letter_and_digits(cpc_list):
    return list(set([code[:3] for code in eval(cpc_list)]))

dataBrevet_cleaned['first_letter_and_digits'] = dataBrevet_cleaned['CPC'].apply(extract_first_letter_and_digits)
dataBrevet_cleaned['first_letter_and_digits']

0             [G02, G06, H04, B23]
1                            [A61]
2                            [D06]
3                            [H01]
4                            [H04]
                   ...            
49995                   [C09, A61]
49996                   [B60, B65]
49997                   [C07, A61]
49998              [G06, G07, H04]
49999    [B33, B08, F01, B01, Y02]
Name: first_letter_and_digits, Length: 50000, dtype: object

In [41]:
# Créer des répertoires pour les fichiers d'entraînement
if not os.path.exists('train_filesLetter'):
    os.makedirs('train_filesLetter')

In [43]:
# Créer les fichiers texte pour chaque lettre spécifique
valid_letters = 'ABCDEFGHY'
for letter in valid_letters:
    with open(f'train_filesLetter/train_{letter}.txt', 'w') as f:
        for _, row in dataBrevet_cleaned.iterrows():
            if letter in row['first_letters']:
                labels = " ".join([f"__label__{label}" for label in row['first_letter_and_digits'] if label.startswith(letter)])
                f.write(f"{labels} {row['FusionClaimDescription']}\n")

In [45]:
# Entraîner et sauvegarder un modèle FastText pour chaque lettre spécifique
for letter in valid_letters:
    train_file = f'train_filesLetter/train_{letter}.txt'
    if os.path.exists(train_file):
        model = fasttext.train_supervised(input=train_file, epoch=50, lr=1.0, wordNgrams=2, bucket=200000, dim=100, loss='softmax')
        model.save_model(f"model_{letter}.bin")

Read 55M words
Number of words:  1160619
Number of labels: 15
Progress: 100.0% words/sec/thread: 4017882 lr:  0.000000 avg.loss:  0.149851 ETA:   0h 0m 0s
Read 61M words
Number of words:  774293
Number of labels: 37
Progress: 100.0% words/sec/thread: 4150257 lr:  0.000000 avg.loss:  0.384881 ETA:   0h 0m 0s 28.2% words/sec/thread: 4123533 lr:  0.717856 avg.loss:  0.784362 ETA:   0h 1m16s
Read 52M words
Number of words:  1440214
Number of labels: 20
Progress: 100.0% words/sec/thread: 3834647 lr:  0.000000 avg.loss:  0.395720 ETA:   0h 0m 0s 68.4% words/sec/thread: 3816780 lr:  0.316293 avg.loss:  0.465831 ETA:   0h 0m31s
Read 4M words
Number of words:  128310
Number of labels: 9
Progress: 100.0% words/sec/thread: 4205055 lr:  0.000000 avg.loss:  0.396862 ETA:   0h 0m 0s
Read 6M words
Number of words:  116386
Number of labels: 7
Progress: 100.0% words/sec/thread: 4411943 lr:  0.000000 avg.loss:  0.220189 ETA:   0h 0m 0s
Read 29M words
Number of words:  337858
Number of labels: 18
Progres

In [53]:
def predict_category_and_explain(text, top_k=3, threshold=0.01):
    # Prédire les premières lettres avec les k labels les plus probables
    first_letter_predictions = model_first_letter.predict(text, k=top_k)
    predicted_first_letters = [label.replace("__label__", "") for label in first_letter_predictions[0]]
    predicted_first_probs = first_letter_predictions[1]
    
    # Prédictions pour les deux premiers chiffres pour chaque lettre prédite
    digits_predictions = {}
    explanations = {}
    for i, letter in enumerate(predicted_first_letters):
        model_path = f"model_{letter}.bin"
        model_specific = fasttext.load_model(model_path)
        digit_predictions = model_specific.predict(text, k=top_k)
        predicted_digits = [label.replace("__label__", "") for label in digit_predictions[0]]
        predicted_probs = digit_predictions[1]
        digits_predictions[letter] = list(zip(predicted_digits, predicted_probs))

        # Obtenir les mots les plus influents pour chaque lettre et chaque chiffre
        explanations[letter] = {}
        first_letter_explanation = explain_prediction(model_first_letter, text, top_k, threshold)
        digit_explanation = explain_prediction(model_specific, text, top_k, threshold)
        explanations[letter]['first_letter'] = first_letter_explanation
        explanations[letter]['digits'] = digit_explanation
    
    return predicted_first_letters, predicted_first_probs, digits_predictions, explanations



In [57]:
# Prédire et expliquer les prédictions pour un texte donné
predicted_first_letters, predicted_first_probs, digits_predictions, explanations = predict_category_and_explain(cleaned_text)

# Affichage des résultats
print(f"Predicted first letters and their probabilities:")
for letter, prob in zip(predicted_first_letters, predicted_first_probs):
    print(f"Letter: {letter}, Probability: {prob:.4f}")

print("\nPredicted digits for each letter and their probabilities:")
for letter, digit_list in digits_predictions.items():
    print(f"\nLetter: {letter}")
    for digit, prob in digit_list:
        print(f"  Digits: {digit}, Probability: {prob:.4f}")

print("\nWords that influenced the prediction:")
for letter in explanations:
    print(f"\nLabel: {letter}")
    print("  First letter influence:")
    for word, importance in explanations[letter]['first_letter'].get(f"__label__{letter}", []):
        print(f"    Word: {word}, Impact: {importance:.4f}")
    print("  Digits influence:")
    for digit in explanations[letter]['digits']:
        for word, importance in explanations[letter]['digits'][digit]:
            print(f"    Digits: {digit.replace('__label__', '')}, Word: {word}, Impact: {importance:.4f}")

Predicted first letters and their probabilities:
Letter: H, Probability: 0.5076
Letter: C, Probability: 0.4430
Letter: B, Probability: 0.0459

Predicted digits for each letter and their probabilities:

Letter: H
  Digits: H01, Probability: 1.0000
  Digits: H05, Probability: 0.0000
  Digits: H10, Probability: 0.0000

Letter: C
  Digits: C22, Probability: 0.9849
  Digits: C25, Probability: 0.0145
  Digits: C23, Probability: 0.0007

Letter: B
  Digits: B32, Probability: 0.9592
  Digits: B23, Probability: 0.0190
  Digits: B22, Probability: 0.0189

Words that influenced the prediction:

Label: H
  First letter influence:
    Word: accord, Impact: 0.0458
    Word: sio2th, Impact: 0.0398
    Word: accord, Impact: 0.0360
    Word: one, Impact: 0.0343
    Word: tin, Impact: 0.0340
    Word: 5, Impact: 0.0331
    Word: 1, Impact: 0.0326
    Word: iron, Impact: 0.0305
    Word: less, Impact: 0.0297
    Word: accord, Impact: 0.0281
  Digits influence:

Label: C
  First letter influence:
    Word: 

Pour avoir le f1 score oobligé cette etape 

In [59]:
# Créer des répertoires pour les fichiers de test
if not os.path.exists('test_filesLetter'):
    os.makedirs('test_filesLetter')

# Créer les fichiers texte pour chaque lettre spécifique
for letter in valid_letters:
    with open(f'test_filesLetter/test_{letter}.txt', 'w') as f:
        for _, row in dataBrevet_cleaned.iterrows():
            if letter in row['first_letters']:
                labels = " ".join([f"__label__{label}" for label in row['first_letter_and_digits'] if label.startswith(letter)])
                f.write(f"{labels} {row['FusionClaimDescription']}\n")


In [79]:
from sklearn.metrics import precision_score, recall_score, f1_score

def evaluate_model(model, test_file):
    y_true = []
    y_pred = []

    # Lire le fichier de test et obtenir les prédictions
    with open(test_file, 'r') as f:
        for line in f:
            labels, text = line.split(' ', 1)
            true_labels = labels.split()
            prediction = model.predict(text.strip())[0]
            y_true.extend(true_labels)
            y_pred.extend(prediction)

    # Nettoyer les étiquettes prédits
    y_pred_clean = [label.replace("__label__", "") for label in y_pred]
    y_true_clean = [label.replace("__label__", "") for label in y_true]

    # Calculer les métriques
    precision = precision_score(y_true_clean, y_pred_clean, average='weighted')
    recall = recall_score(y_true_clean, y_pred_clean, average='weighted')
    f1 = f1_score(y_true_clean, y_pred_clean, average='weighted')

    return precision, recall, f1

# Évaluer les performances pour les deux premiers chiffres de chaque lettre prédite
precision_digits = []
recall_digits = []
f1_digits = []

for letter in valid_letters:
    test_file = f'test_filesLetter/test_{letter}.txt'
    model_specific = fasttext.load_model(f'model_{letter}.bin')
    precision, recall, f1 = evaluate_model(model_specific, test_file)
    precision_digits.append(precision)
    recall_digits.append(recall)
    f1_digits.append(f1)

# Calculer les métriques moyennes
average_precision_digits = sum(precision_digits) / len(precision_digits)
average_recall_digits = sum(recall_digits) / len(recall_digits)
average_f1_digits = sum(f1_digits) / len(f1_digits)

print(f"Précision (Digits) : {average_precision_digits:.4f}")
print(f"Rappel (Digits) : {average_recall_digits:.4f}")
print(f"F1-score (Digits) : {average_f1_digits:.4f}")


Précision (Digits) : 0.9320
Rappel (Digits) : 0.9239
F1-score (Digits) : 0.9240


In [81]:
import streamlit as st
import fasttext
import streamlit as st
import fasttext
from utils import clean_text, basic_cleaning, remove_stopwords, predict_category_and_explain,explain_prediction
import os


# Interface utilisateur
st.title("Prédiction de Catégories de Brevets")
user_input = st.text_area("Entrez le texte du brevet ici:")
if st.button("Prédire"):
    if user_input:
        predicted_first_letter, _ = model_first_letter.predict(cleaned_text, k=1)
        predicted_first_letter = predicted_first_letter[0].replace("__label__", "")
        
        st.write(f"Première lettre prédite : {predicted_first_letter}")
        
        if predicted_first_letter in models_digits:
            model_digits = models_digits[predicted_first_letter]
            predicted_digits, digit_probs = model_digits.predict(cleaned_text, k=3)
            predicted_digits = [digit.replace("__label__", "") for digit in predicted_digits]
            
            st.write(f"Chiffres prédits : {', '.join(predicted_digits)}")

            # Explication pour la première lettre
            word_importances_first_letter = explain_prediction(model_first_letter, cleaned_text)
            st.write("Mots influents pour la première lettre :")
            for label in word_importances_first_letter:
                st.write(f"\nLabel: {label}")
                for word, importance in word_importances_first_letter[label]:
                    st.write(f"  Mot: {word}, Impact: {importance:.4f}")

            # Explication pour les chiffres prédits
            word_importances_digits = explain_prediction(model_digits, cleaned_text)
            st.write("Mots influents pour les chiffres prédits :")
            for label in word_importances_digits:
                st.write(f"\nLabel: {label}")
                for word, importance in word_importances_digits[label]:
                    st.write(f"  Mot: {word}, Impact: {importance:.4f}")
        else:
            st.write("Modèle pour les deux premiers chiffres non trouvé.")
    else:
        st.write("Veuillez entrer un texte pour la prédiction.")